# Project Milestone2 - Group Concatsanddogs

# The use of women's rights and gender equality rhetoric in the US

## Header

This notebook is organized followings the different steps used in our pipeline. We first create a list of keywords using [web scraping](#Webscraping) and a personal list of keywords. 
With this list of keywords we [select](#Dataset-selection-from-Quotebank-database) a subset of the Quotebank database. This subset will be our starting dataset for our project. 

## General librairies

In [1]:
import pandas as pd
import numpy as np

## Webscraping

AMINA TO ADD

### Libraries

### Functions

## Dataset selection from Quotebank database

### Librairies

In [2]:
import os 


### Functions

In [3]:
#Processing on chunk
#Input
#Output
def process_chunk(chunk, vocabulary):
    print(f'Processing chunk with {len(chunk)} rows')
    #print(chunk.columns)
    occurences = np.zeros(len(vocabulary))
    for index, word in enumerate(vocabulary):
        occurences[index] = np.sum(chunk['quotation'].str.contains(word)) 
    return occurences

#Select quotes containing keywords
def select_quotes_chunk(chunk, keywords):
    print(f'Processing chunk with {len(chunk)} rows')
    return chunk[chunk['quotation'].str.contains('|'.join(keywords))]

#Use the selection function on each chunk of the full dataset 
def select_quotes_one_year(path_to_file, vocabulary, chunksize = 10 ** 4):
    with pd.read_json(path_to_file, lines=True, compression='bz2', chunksize=chunksize) as df_reader:
        for index, chunk in enumerate(df_reader):
            if not index==0:
                selected_df = pd.concat([selected_df, select_quotes_chunk(chunk, vocabulary)])
            else: 
                selected_df = select_quotes_chunk(chunk, vocabulary)
    return selected_df

#Use the selection function on each chunk of the full dataset 
#Dumps the selected quotes into a new json file
def select_and_dump(path_to_file, vocabulary, chunksize = 10 ** 4, year = 'replace_me'):
    with pd.read_json(path_to_file, lines=True, compression='bz2', chunksize=chunksize) as df_reader:
        for index, chunk in enumerate(df_reader):
            #Dump selected quotes
            selected_df = select_quotes_chunk(chunk, vocabulary)
            pickle_file_name = year + '_chunk_' + str(index) + '.pkl'
            selected_df.to_pickle('files/'+pickle_file_name)
            #if not index==0:
                #selected_df = pd.concat([selected_df, select_quotes_chunk(chunk, vocabulary)])
            #else: 
               # selected_df = select_quotes_chunk(chunk, vocabulary)
    return selected_df


import random, string

def randomword(length):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(length))

### Constants 

In [12]:
DATA_FOLDER = 'data/' # ../ doesn't work for me for some reason
QUOTEBANK_2020 = DATA_FOLDER+ "quotes-2020.json.bz2"
QUOTEBANK_2019 = DATA_FOLDER+ "quotes-2019.json.bz2"
QUOTEBANK_2017 = DATA_FOLDER+ "quotes-2017.json.bz2"
QUOTEBANK_2015 = DATA_FOLDER+ "quotes-2015.json.bz2"
QUOTEBANK_2018 = DATA_FOLDER+ "quotes-2018.json.bz2"
QUOTEBANK_2016 = DATA_FOLDER+ "quotes-2016.json.bz2"

KEYWORDS_LIST = ('women\'s right','Equal opportunities','Equal rights','Equal status','equal pay',
              'gender gap','Gender discrimination','Gender equality','Sexual harrassment',
              'Women empowerment','women victim','women immigration','Women emancipation',
              'women\'s participation','Western women','non-western woman','Muslim women',
              'Equal wages','Gender equality','gender equity','Men and women','women and men',
              'women oppression','niqab ban','struggle of girls','struggle of women','war against women',
              'oppression of girls','oppression of women','women oppression','women\'s opression','liberate women',
              'religious oppresion','abuse of women','Male oppression','Female oppression','Exploitation of women',
              'Indigenous women','Patriarchal culture','gender equality','child care','men pay','percentage men',
              'pay percentage','sexual harassment','women girls','girls women',
              'rates women','women according','female mayors','share women','women movement',
              'see women','gender stereotypes','gender gap',
              'women representation','sex discrimination','states women','lose weight',
              'women rights','woman time',
              'based gender',
              'proportional electoral','female candidates','gender-based violence','entirely female','cities female')

### Select and pickle of quotes of interest

Note: This code has to be run once to create the pickle files containing the quotes of interest. For futher use, the dataframe is direcly loaded from the pickle files.

In [15]:

%time QOI_TEST_DF_Dumped = select_and_dump(DATA_FOLDER+ "quotes-2019-nytimes.json.bz2", KEYWORDS_LIST, 10 ** 4, year = 'qb_TEST') 
QOI_TEST_DF_Dumped.to_pickle('files/'+'TEST_selected.pkl')

Processing chunk with 10000 rows
Processing chunk with 10000 rows
Processing chunk with 10000 rows
Processing chunk with 10000 rows
Processing chunk with 10000 rows
Processing chunk with 10000 rows
Processing chunk with 10000 rows
Processing chunk with 10000 rows
Processing chunk with 10000 rows
Processing chunk with 10000 rows
Processing chunk with 10000 rows
Processing chunk with 10000 rows
Processing chunk with 10000 rows
Processing chunk with 10000 rows
Processing chunk with 10000 rows
Processing chunk with 10000 rows
Processing chunk with 10000 rows
Processing chunk with 10000 rows
Processing chunk with 10000 rows
Processing chunk with 10000 rows
Processing chunk with 7527 rows
Wall time: 17.1 s


In [13]:
#The two first have been directly dumped into multiples pickle files
#%time QOI_2020_DF_Dumped = select_and_dump(QUOTEBANK_2020, KEYWORDS_LIST, 10 ** 6, year = 'qb_2020') 
#QOI_2020_DF_Dumped.to_pickle('files/'+'2020_selected.pkl')

KeyboardInterrupt: 

NameError: name 'QOI_2020_DF_Dumped' is not defined

In [ ]:
#%time QOI_2019_DF_Dumped = select_and_dump(QUOTEBANK_2019, KEYWORDS_LIST, 10 ** 6, year = 'qb_2019')
#QOI_2019_DF_Dumped.to_pickle('files/'+'2019_selected.pkl')

#%time QOI_2018_DF = select_quotes_one_year(QUOTEBANK_2018,KEYWORDS_LIST,10 ** 6, year = 'qb_2018')
#QOI_2018_DF.to_pickle('files/'+'2018_selected.pkl')

#%time QOI_2017_DF = select_quotes_one_year(QUOTEBANK_2017,KEYWORDS_LIST,10 ** 6, year = 'qb_2017')
#QOI_2017_DF.to_pickle('files/'+'2017_selected.pkl')

#%time QOI_2016_DF = select_quotes_one_year(QUOTEBANK_2016,KEYWORDS_LIST,10 ** 6, year = 'qb_2016')
#QOI_2016_DF.to_pickle('files/'+'2016_selected.pkl')

#%time QOI_2015_DF = select_quotes_one_year(QUOTEBANK_2015,KEYWORDS_LIST,10 ** 6, year = 'qb_2015')
#QOI_2015_DF.to_pickle('files/'+'2015_selected.pkl')

### Load dataframe of selected quotes

In [21]:
'files/'+ files

'files/TEST_selected.pkl'

In [28]:
# Concatenate the dataframes obtained from each of the pickle file. 
# files = ('2015_selected.pkl','2016_selected.pkl','2017_selected.pkl','2018_selected.pkl','2019_selected.pkl','2020_selected.pkl')
# df = pd.concat([pd.read_pickle('files/'+ fp) for fp in files], ignore_index=True)
df = pd.read_pickle('files/TEST_selected.pkl')

In [29]:
df.head()

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
200262,2019-07-02-019858,He has spoken up for women's issues and women'...,Serena Williams,[Q11459],2019-07-02 23:04:15,1,"[[Serena Williams, 0.8938], [None, 0.0997], [A...",[https://www.nytimes.com/2019/07/02/sports/ser...,E
201055,2019-03-13-126346,"With this cruel sentence, the Iranian authorit...",Kumi Naidoo,[Q1363729],2019-03-13 16:55:11,4,"[[Kumi Naidoo, 0.93], [None, 0.0561], [Nasrin ...",[https://www.seattletimes.com/opinion/two-wome...,E
201211,2019-05-02-052509,It basically said we needed to address sexual ...,None,[],2019-05-02 20:27:34,1,"[[None, 0.5979], [Marcia McNutt, 0.4021]]",[http://www.nytimes.com/2019/05/02/science/nat...,E
203171,2019-08-29-121813,You had several female candidates who were all...,Randi Weingarten,[Q7291716],2019-08-29 19:47:38,1,"[[Randi Weingarten, 0.7464], [None, 0.1963], [...",[http://mobile.nytimes.com/2019/08/29/us/polit...,E
203360,2019-03-25-105644,We're doing all these grown-up things like mak...,Bill Keller,"[Q3640044, Q4909680, Q862237]",2019-03-25 21:30:55,1,"[[Bill Keller, 0.8768], [None, 0.1232]]",[http://www.nytimes.com/2019/03/25/business/ch...,E


In [30]:
print(f'The dataframe has {len(df)} entries')

The dataframe has 8 entries


In [31]:
pd.options.display.max_colwidth = 100
df.head()['quotation']

200262                 He has spoken up for women's issues and women's rights, especially in tennis, forever.
201055    With this cruel sentence, the Iranian authorities appear to be seeking to make an example of Nas...
201211    It basically said we needed to address sexual harassment as seriously as other forms of academic...
203171                   You had several female candidates who were all bringing their own brand of feminism.
203360    We're doing all these grown-up things like making sure that we have policies on paid leave and s...
Name: quotation, dtype: object